In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

In [2]:
data=pd.read_csv("/Users/mona/Documents/ann classification1/Churn_Modelling.csv")

In [3]:
# Step 3: Drop Unnecessary Columns
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [9]:
# Step 4: Label Encode 'Gender'
le = LabelEncoder()
data['Gender'] = le.fit_transform(data['Gender'])  # Female = 0, Male = 1
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [11]:
onehot_encoder_geo=OneHotEncoder()
geo_encoder=onehot_encoder_geo.fit_transform(data[["Geography"]])
geo_encoder

<10000x3 sparse matrix of type '<class 'numpy.float64'>'
	with 10000 stored elements in Compressed Sparse Row format>

In [13]:
onehot_encoder_geo.get_feature_names_out(["Geography"])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [15]:
geo_encoder_df = pd.DataFrame(
    geo_encoder.toarray(),
    columns=onehot_encoder_geo.get_feature_names_out(["Geography"])
)

In [17]:
# Step 5: Concatenate with original data (drop 'Geography' column first)
data = pd.concat([data.drop("Geography", axis=1), geo_encoder_df], axis=1)


In [19]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [21]:
#save the encoder and scaler in pickle file
import pickle
with open("onehot_encoder_geo","wb")as file:
    pickle.dump(onehot_encoder_geo,file)


In [23]:
import pickle

# Assuming `le` is your LabelEncoder object
with open('le.pkl', 'wb') as file:
    pickle.dump(le, file)


In [25]:
#divide the dataset 
X=data.drop("Exited",axis=1)
y=data['Exited']

In [27]:
#spliting into train test spilt
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

##scaling 
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [29]:
X_train


array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [31]:
#save the scaler into pickle
#A pickle file (.pkl) is used to save a Python object (like a model, DataFrame, or list) so you can load it later without redoing all the work.
with open("scaler","wb")as file:
    pickle.dump(scaler,file)

### ANN implementation

#basically its an seq layer ie its interconnected 
#for hidden layer we usually need to use an dense
##next is activation funcs it can be anything like sigmoid for single class ,softmax for multiclass bt relu is preferable
#optimizer basicallly used in the back propogation for to reduce the loss/cost function--updating the weights
##loss function
#metrics
##training info save into logs ==folder____tensorflow to visulaise the logs ithe way we understand



In [35]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime
#Sequential: To build your ANN layer by layer.
#Dense: For fully connected layers in the network.
#EarlyStopping: To stop training when performance stops improving.

In [37]:
X_train[1].shape

(12,)

In [63]:
###buikd ANN model that input_shape is nes for only the first hidden layer not nes for 2nd at all
# Build the ANN model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),  # First hidden layer with input shape
    Dense(32, activation='relu'),                                   # Second hidden layer
    Dense(1, activation='sigmoid')                                  # Output layer for binary classification
])


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [65]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [67]:
import tensorflow
opt=tensorflow.keras.optimizers.Adam(learning_rate=0.01)#to give own lr---speed /convergence rate ++ any opti

In [69]:
##so here in hl1 64 neurons r there hl2-32 and hl3 or op we have -1 neuron  and param is combination of w and bias 12*64 = w + 64 bias 
#compile #loss we find by predicted vs actual; by the loss we need to reduce
model.compile(optimizer=opt,loss='binary_crossentropy',metrics=["accuracy"])

In [71]:
#set up tensorbrd
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
log_dir="logs/fit/"+ datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)#main aim of tensorbrd is to vis the logs

In [73]:
##setup earlystop eg 100 epoch ==at 20th epoch the modek trained well means we can stop 1epoch = 1 round of frdp and bwrdp
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [75]:
####training the model
history=model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100,
                  callbacks=[tensorflow_callback,early_stopping_callback])## according to the patience lvl epochs happend 
#it waits for 10 and run till 18 Patience = 10 means: "Wait 10 bad (non-improving) epochs before quitting."
#So if last improvement was at epoch 8, training stops at epoch 18.

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 6s 14ms/step - accuracy: 0.8161 - loss: 0.4260 - val_accuracy: 0.8550 - val_loss: 0.3482
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8637 - loss: 0.3508 - val_accuracy: 0.8575 - val_loss: 0.3488
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.8627 - loss: 0.3400 - val_accuracy: 0.8560 - val_loss: 0.3570
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.8563 - loss: 0.3503 - val_accuracy: 0.8610 - val_loss: 0.3401
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.8638 - loss: 0.3370 - val_accuracy: 0.8595 - val_loss: 0.3493
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.8610 - loss: 0.3344 - val_accuracy: 0.8595 - val_loss: 0.3443
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8605 - loss: 0.3360 - val_accuracy: 0.8545 - val_loss: 0.3433
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.8615 - loss: 0.3357 - val_a

In [77]:
model.save('model.keras', include_optimizer=False)


In [53]:
#load tnsbrd extension
%load_ext tensorboard

In [59]:
%tensorboard --logdir logs/fit  ##
#launches TensorBoard to visualize your model's training logs (like loss & accuracy) stored in the logs/fit folder.

## so here graphs is failed due to we used seq 
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Dense

inputs = Input(shape=(X_train.shape[1],))
x = Dense(64, activation='relu')(inputs)
x = Dense(32, activation='relu')(x)
outputs = Dense(1, activation='sigmoid')(x)

model = Model(inputs, outputs)


to fix the Graph tab issue, use the Functional API instead of Sequential:


In [ ]:
###load the pickle file for prediction
